In [2]:
import os, logging

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow_hub").setLevel(logging.CRITICAL)

import tensorflow as tf
import tensorflow_hub as hub

In [3]:
#import tensorflow as tf
import environment
import math
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

pygame 2.0.1 (SDL 2.0.14, Python 3.7.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
env = environment

n_episodes = 1000
n_win_ticks = 3000

gamma = 0.999
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01  
alpha_decay = 0.01

batch_size = 500

memory = deque(maxlen = 10000)

SHOW_EVERY = 20
PLOT_EVERY = 20
RESET_EVERY = 20
show =  False

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(24, input_dim = 11, activation = 'relu'))
model.add(Dense(48, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'mse', optimizer = Adam(lr = alpha, decay = alpha_decay))

In [11]:
def save(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))   
    
def choose_action(state, epsilon):
    if np.random.random() <= epsilon:
        return env.sample() 
    else: 
        return np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1)*epsilon_decay)))

def replay(batch_size, epsilon):
    
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in minibatch:  
        
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma*np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
    
    model.fit(np.array(x_batch), np.array(y_batch), batch_size = len(x_batch), verbose = 0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [13]:
def run():
    ep = []
    ticks = []
    # make the environment
    env = environment
    env.reset()
    env.show = show
    # run the loop for n_episodes
    for e in range(n_episodes):
        global memory
        # Reset memory
        if e % RESET_EVERY == 0:
            memory = []
        # Reset the environment
        env.reset()
        done = False
        # First state
        state = env.get_state()
        # Reset the score counter
        i = 0
        # Run the loop - algorithm
        while not done:
            # get the action according the state.
            action = choose_action(state, epsilon)
            # Make the move
            env.step(action)
            # Get the next state from environment
            next_state = env.get_state()
            # Get the reward
            reward = env.get_reward()
            # Check if the programm has done
            done = env.done
            # Save the current situation
            save(state, action, reward, next_state, done)
            # Update the state
            state = next_state
            # Raise up the score
            i += 1
        
        print("Episode {} run for {} ticks".format(e, i))
        replay(batch_size, get_epsilon(e))
        
        ep.append(e)
        ticks.append(i)
        
        plt.plot(ep, ticks)
        if e % PLOT_EVERY == 0 and e > 1:
            plt.show()
        
        if show == True:
            if e % SHOW_EVERY == 0:
                env.show = True
            else:
                env.show = False

In [ ]:
run()

Episode 0 run for 37 ticks
Episode 1 run for 31 ticks
Episode 2 run for 23 ticks
Episode 3 run for 24 ticks
Episode 4 run for 46 ticks
Episode 5 run for 27 ticks
Episode 6 run for 28 ticks
Episode 7 run for 40 ticks
Episode 8 run for 49 ticks
Episode 9 run for 34 ticks
Episode 10 run for 27 ticks
Episode 11 run for 32 ticks
Episode 12 run for 22 ticks
Episode 13 run for 28 ticks
Episode 14 run for 23 ticks
Episode 15 run for 28 ticks
Episode 16 run for 24 ticks
Episode 17 run for 45 ticks


In [ ]:
model.save('model1.h5')
new_model = tf.keras.models.load_model('model1.h5')

In [7]:
#model = tensorflow.keras.models.load_model('model1.h5')
def test(model):
    i = 0
    env = environment
    env.show = True
    env.reset()
    done = False
    while not done:
        state = env.get_state()
        action = np.argmax(model.predict(state))
        env.step(action)
        done = env.done
        i += 1
    print("The total ticks was {}".format(i))

In [ ]:
test()